# Neo4j Map of SP

In [1]:
from py2neo import Graph, Node, Relationship

Start your neo4j graph server
- port 7687 is where the data stuff goes through
- port 7474 is what you'll use to view the graph (localhost:7474)

### Note: The password is configured on your own but you can usethe same password (ILikeYuri) because why not

In [2]:
graph = Graph("bolt://localhost:7687", auth = ("neo4j", 
                                       "ILikeYuri"))

In [3]:
from graph_nodes import graph_nodes, relation_nodes

In [4]:
def ResetNeo():
    graph.run('''
      MATCH (n)
      OPTIONAL MATCH (n)-[r]-()
      WITH n,r LIMIT 50000
      DELETE n,r
      RETURN count(n) as deletedNodesCount
    ''')
ResetNeo()

In [5]:
for node_type in graph_nodes:
    for node_name, node in node_type.items():
        graph.create(node)

In [6]:
for rel_type in relation_nodes:
    for rel in rel_type:
        graph.create(rel)

In [7]:
def get_node(name: str):
    query = f"match (n) where n.name = '{name}' return n"
    node = graph.evaluate(query)
    return node

In [8]:
# Get the label of that node
def get_node_label(node: Node):
    label = str(node.labels).replace(':', '')
    return label

In [9]:
# returns list of all paths (as tuple of relationship objects)
def get_paths(node_a_name: str, node_b_name: str):
    node_a = get_node(node_a_name)
    node_a_label = get_node_label(node_a)
    
    node_b = get_node(node_b_name)
    node_b_label = get_node_label(node_b)
    
    if node_a == None or node_b == None:
        print("Node not found.")
    
    param_a = '{' + f"name: '{node_a_name}'" + '}'
    param_b = '{' + f"name: '{node_b_name}'" + '}'
    query_string = f"match p=(:{node_a_label}{param_a})-[*]-(:{node_b_label}{param_b}) return p"
    
    path_list = []
    path_obj_list = [] # contains real path objects
    result = graph.run(query_string).data()

    for path in result:
        for x, y in path.items():
            # print(type(y), y)
            path_list.append(y.relationships)
            path_obj_list.append(y)
    return path_list, path_obj_list

In [10]:
def get_path_dist(path: tuple):
    total_dist = 0
    for rel in path:
        dist = rel['distance']
        total_dist += dist
    return total_dist

In [11]:
def get_shortest_path_dist(path_list: list, real_paths_list):
    all_path_dist = []
    for path_tuple in path_list:
        # print(path_tuple)
        all_path_dist.append(get_path_dist(path_tuple))
    # all_path_dist.append(7)
    # print(all_path_dist)
    shortest_path_ind = all_path_dist.index(min(all_path_dist))
    return real_paths_list[shortest_path_ind]

In [33]:
def walk_path(path):
    
    count = 0
    total_dist = 0
    path_size = len(path.nodes) + len(path.relationships)
    while count < path_size:
        try:
            node_name = path.nodes[count]['name']
            if count == len(path.nodes) - 1:
                print(f"You have reached {node_name} :)")
                print(f"Total Distance: {total_dist} metres")
                return
            print(f"At <{node_name}>,")
            print(path.relationships[count]['description'])
            path_dist = path.relationships[count]['distance']
            if path_dist != 0:
                print(f"This path is around {path_dist} metres")
            #print(path.nodes[count]['name'])
            #print(path.relationships[count]['distance'])
            total_dist += path.relationships[count]['distance']
            print()
            count += 1
        except IndexError:
            return

In [34]:
# main function
def walk_shortest_path(node_a_name: str, node_b_name: str):
    print(f"Fetching directions from <{node_a_name}> to <{node_b_name}>\n")
    paths, real_paths_list = get_paths(node_a_name, node_b_name)
    shortest_path = get_shortest_path_dist(paths, real_paths_list)
    walk_path(shortest_path)

In [35]:
walk_shortest_path('T1', "Food Court 6")

Fetching directions from <T1> to <Food Court 6>

At <T1>,
Climb up to level 2 and access the building through the bridge
This path is around 15.81 metres

At <T2>,
Climb up to level 2 and access the building through the bridge
This path is around 11.13 metres

At <T3>,
Walk to the next nearest turn and turn a left then a right then keep walking straight
This path is around 18.7 metres

At <T5>,
Turn right from the intersection of the bridge and walk to the end of the corridor

At <SPavillion>,
Walk along the classrooms and turn right before turning a left
This path is around 7.02 metres

At <Colours>,
Climb up to level 2 and access the building through the bridge
This path is around 16.34 metres

At <The Sanctuary>,
Turn left. Walk right, walk until you see it
This path is around 20.27 metres

At <T10>,
Walk to the next nearest turn and turn a left then a right then keep walking straight
This path is around 9.74 metres

At <Poly Centre>,
Turn left. Walk right, walk until you see it
Thi

### Getting Nodes

In [15]:
def get_building(name: str):
    b = get_node(name)
    print(f"Building: {name}, School: {b['school']}")
    print(b['description'])

In [16]:
get_building("T19")

Building: T19, School: SoC
This building is filled with classrooms.


In [17]:
 def get_facility(name: str):
    f = get_node(name)
    print(f"Facility: {name}, Type: {f['type']}")
    print(f['description'])

In [18]:
get_facility("Hilltop Library")

Facility: Hilltop Library, Type: library
blue


### Get relations

In [20]:
def get_all_node_rels(node_name: str, rel_name: str, node_type: str = 'building'):
    params = '{' + f"name: '{node_name}'" + '}'
    query = f"match (:{node_type}{params})-[r]-() return r"
    # print(query)
    results = graph.run(query).data()
    
    lots_available = 0
    filtered_rels = []
    for r in results:
        r = r['r']
        #print(r.type)
        #print(type(r))
        rel_label = str(type(r)).split('.')[-1].replace("'>", '')
        if rel_label == rel_name:
            filtered_rels.append(r)
    return filtered_rels

In [23]:
def get_lots_available(parking_lot_name: str):
    lots = get_all_node_rels(parking_lot_name, "HAS_PARKING_LOT")
    print(f"<{parking_lot_name}>")
    print(f"Available lots: {len(lots)}")

In [24]:
get_lots_available("Parking Lot A")

<Parking Lot A>
Available lots: 20


In [25]:
# def get_relations(rel_name: str):
#     query = f"match ()-[r:{rel_name.upper()}]-() return r"
#     result = graph.run(query).data()
#     print(result)
#     print(type(result) + '\n')

In [26]:
# get_relations("walk")

In [27]:
def get_events():
    print("Fetching current events in SP...\n")
    query = "match (n:event) return n"
    results = graph.run(query).data()
    for ind, e in enumerate(results):
        e = e['n']
        print(ind + 1, e['name'])
        print(e['description'])
        
        rels = get_all_node_rels(e['name'], "HELD_IN", 'event')
        held_in_node = rels[0].nodes[-1]['name']
        print(f"It is held in {held_in_node}")
        print()

In [28]:
get_events()

Fetching current events in SP...

1 SmartPoly
This is the smartpoly hackathon 2019.
It is held in T14

2 WSS Training
WSS Training for WSS 2020. Held from 9am to 3pm
It is held in T21



In [ ]:
# t19_node = Node("Building", name = 'T19', heck = 'yeet')

In [ ]:
# graph.create(t19_node)

In [ ]:
# fc6 = Node("FoodCourt", name = "Food Court 6", meh = "heck")

In [ ]:
# graph.create(fc6)

In [ ]:
# connected_to_rel = Relationship(t19_node, "Connected_to", fc6, distance = 7, descrip = "VER LONG")
# graph.create(connected_to_rel)